### Import required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from CartPole import CartPole
from CartPole_GPS import CartPole_GPS
from ilqr.dynamics import constrain
from copy import deepcopy
from EstimateDynamics import local_estimate

### Formulate the iLQR problem

In [2]:
'''
1 - dt = time step
2 - N = Number of control points in the trajectory
3 - x0 = Initial state
4 - x_goal = Final state
5 - Q = State cost
6 - R = Control cost
7 - Q_terminal = Cost at the final step
8 - x_dynamics array stores the information regarding system. 
    x_dynamics[0] = m = mass of the pendulum bob 
    x_dynamics[1] = M = mass of the cart 
    x_dynamics[2] = L = length of the massless rod 
    x_dynamics[3] = g = gravity 
    x_dynamics[4] = d = damping in the system
'''
dt = 0.005
N = 500  # Number of time steps in trajectory.
x_dynamics = np.array([1, 5, 2, 9.80665, 1]) # m=1, M=5, L=2, g=9.80665, d=1
x0 = np.array([-3.0, 0.0, 0.1, 0.0])  # Initial state
x_goal = np.array([2.0, 0.0, 0.0, 0.0])
# Instantenous state cost.
Q = np.eye(5)
Q[2, 2] = 10
Q[3, 3] = 10
Q[4, 4] = 100
# Terminal state cost.
Q_terminal = 100 * np.eye(5)
# Instantaneous control cost.
R = np.array([[1.0]])

### iLQR on Cart Pole

In [3]:
cartpole_prob = CartPole(dt, N, x_dynamics, x0, x_goal, Q, R, Q_terminal)
xs, us = cartpole_prob.run_IterLinQuadReg()

iteration 0 accepted 721665.7474373202 [7.54851063 0.48632489 0.19374494 4.21301541]
iteration 1 accepted 665131.7600970949 [1.94767955 1.33503646 2.9912245  4.48439422]
iteration 2 accepted 655860.4649839161 [7.81812795 2.47307106 1.79000883 3.33968554]
iteration 3 accepted 630011.4256804134 [ 2.66920372  0.29941602 -0.134569   -0.04770233]
iteration 4 accepted 626423.6649000604 [3.69632606 0.25122802 0.46757667 1.2939389 ]
iteration 5 accepted 618169.2717320804 [ 2.2109785  -0.14777124 -0.08684245  0.37143337]
iteration 6 accepted 614745.7150416166 [ 2.1742186  -0.42281199 -0.20058166  0.21732799]
iteration 7 accepted 611568.1909383517 [ 2.28762541 -0.39005038 -0.15198728  0.51485791]
iteration 8 accepted 608611.7141859714 [ 2.11716138 -0.4859255  -0.26265114  0.39083614]
iteration 9 accepted 606059.932959753 [ 2.27830416 -0.5229117  -0.27815375  0.62772499]
iteration 10 accepted 603624.9514343122 [ 2.16520681 -0.58063258 -0.33007885  0.64155818]
iteration 11 accepted 601366.08572464

iteration 92 accepted 454105.9441669732 [ 2.0299098  -0.3758577  -0.52995579 -0.15511045]
iteration 93 accepted 453686.2645436086 [ 2.03420759 -0.39241259 -0.53314141 -0.10191507]
iteration 94 accepted 453273.3849596563 [ 2.03903004 -0.40944469 -0.53641907 -0.04604189]
iteration 95 accepted 452866.27942017245 [ 2.04437285 -0.42691808 -0.54003306  0.01234369]
iteration 96 accepted 452463.87119103293 [ 2.05023714 -0.44480297 -0.5442155   0.07315639]
iteration 97 accepted 452065.02465565514 [ 2.05662992 -0.46307469 -0.54919236  0.13638899]
iteration 98 accepted 451668.5300198994 [ 2.06356464 -0.48171212 -0.55519065  0.20211162]
iteration 99 accepted 451273.0799131597 [ 2.07106175 -0.50069566 -0.56244658  0.27047349]


In [ ]:
# State matrix split into individual states. For plotting and analysing purposes.
t = np.arange(N + 1) * dt
x = xs[:, 0] # Position
x_dot = xs[:, 1] # Velocity
theta = np.unwrap(cartpole_prob.deaugment_state(xs)[:, 2])  # Theta, makes for smoother plots.
theta_dot = xs[:, 3] # Angular velocity

### Simulate the real system and generate the data
Cost matrices, initial position and goal position will remain same as the above problem. As it indicates one policy. But still the initial positions and goal positions must be passed explicitly to the function. But you don't need to pass cost matrices (assume penalty on the system is same), this is just used to use to calculate the cost of the trajectory. Correct control action must be passed. Parameter gamma indicates how much of original data you want to keep

Variance of the Gaussian noise will be taken as input from a Unif(0, var_range) uniform distribution. Inputs: x_initial, x_goal, u, n_rollouts, pattern='Normal', pattern_rand=False, var_range=10, gamma=0.2, percent=20

Pattern controls how the control sequence will be modified after applying white Guassian noise (zero mean).
- Normal: based on the correction/mixing parameter gamma generate control (gamma controls how much noise we want).
- MissingValue: based on the given percentage, set those many values to zero (it is implicitly it uses "Normal" generated control is used). 
- Shuffle: shuffles the entire "Normal" generated control sequence.
- TimeDelay: takes the "Normal" generated control and shifts it by 1 index i.e. one unit time delay.
- Extreme: sets gamma as zeros and generates control based on only noise.

If 'pattern_rand' is 'True' then we don't need to send the explicitly, it will chose one randomly for every rollout (default is 'False'). If you want to chose specific pattern then send it explicitly. 

In [ ]:
x_rollout, u_rollout, local_policy, x_gmm, cost = cartpole_prob.gen_rollouts(x0, x_goal, us, n_rollouts=10, pattern_rand=True, var_range=10, gamma=0.2, percent=20)

### Local system dynamics/model estimate
loca_estimate: function takes the states (arranged in a special format, [x(t), u(t), x(t+1)]), no. of gaussian mixtures and no.of states.

In [ ]:
model = local_estimate(x_gmm, components=5, NoOfstates=5)
A, B, C = model.estimate(N=N)

### iLQR on estimated model
Here system dynamics is specified in a special way. We give the A, B, C matrices as input. These matrices comes from GMM and GPS theory. They are the mean/expected trajectory followed by the states which is represented by the mean & covariance (A, B, C) matrices of a Gaussian. Remaining all properties of the iLQR problem remains the same (cost, initial & goal state, time steps). 

In [ ]:
x_traj,u_traj = cartpole_prob.run_IterLinQuadReg_matrix(A, B, C)

### Plot

In [ ]:
# Control sequence
plt.plot(np.arange(us.shape[0]), us, 'r.', label='Original')
plt.plot(np.arange(us.shape[0]), u_rollout[0:N], 'b.', label='Corrupted')
plt.plot(np.arange(us.shape[0]), u_traj, 'g.', label='Estimated')
plt.xlabel('Time steps')
plt.ylabel('U')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(xs.shape[0]), xs[:, 2], 'r.', label='Original')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout)[0:N+1, 2], 'b.', label='Corrupted')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 2], 'g.', label='Estimated')
plt.xlabel('Time steps')
plt.ylabel('Theta')
plt.legend()
plt.show()

In [ ]:
plt.plot(np.arange(xs.shape[0]), xs[:, 0], 'r.', label='Original')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_rollout)[0:N+1, 0], 'b.', label='Corrupted')
plt.plot(np.arange(xs.shape[0]), cartpole_prob.deaugment_state(x_traj)[:, 0], 'g.', label='Estimated')
plt.xlabel('Time steps')
plt.ylabel('Pos')
plt.legend()
plt.show()

### Mean and Variance 

In [ ]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[1, 2, 3], [4, 5, 6]])
c = np.array([[1, 2], [3, 4], [5, 6]])
d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(np.block([[a, b], [c, d]]))